In [ ]:
import datetime
import requests
import pandas as pd
import numpy as np
from pathlib import Path

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('dark_background')


In [ ]:
SAVE_IMAGES = True

def plot(axes, title=None, file=None, ylim=None, pppp=plt, xticks=None):
    if title: axes.set_title(title)
    axes.legend(loc="upper left")
    axes.set_xlim(None, None)
    axes.autoscale()
    pppp.tight_layout()
    if ylim: 
        if type(ylim) == int: pppp.ylim(ylim)
        else: pppp.ylim(ylim[0], ylim[1])
    # print(f"ylim={ylim} type={type(ylim)} int={type(ylim)==int}")
    if xticks: pppp.xticks(xticks, rotation=90)
    if SAVE_IMAGES and file: pppp.savefig(file)
    return pppp

def f(valor, plus=False):
    if valor is None: return None
    valor = valor if type(valor) == int else float(valor)
    r = format(valor, ",").replace(".","!").replace(",",".").replace("!",",")
    return f"+{r}" if plus and valor > 0 else r


In [ ]:
# População residente em PT final 2019, via
# https://www.ine.pt/xportal/xmain?xpid=INE&xpgid=ine_indicadores&contecto=pi&indOcorrCod=0008273&selTab=tab0
# Coerente com a soma da população dos concelhos, vide POP_ARS abaixo
POP_PT = 10295909

# data_concelhos_new.csv:
# df[df.data == '11-11-2020'][["ars", "population"]].groupby('ars').sum()
POP_ARS = {
    'norte':   3568835,  # 3.57M
    'centro':  1650394,  # 1.65M
    'lvt':     3674534,  # 3.67M
    'alentejo': 466690,  # 0.46M
    'algarve':  438406,  # 0.44M
    'acores':   242796,  # 0.24M
    'madeira':  254254,  # 0.25M
}
POP_IDADE = {
    '0_9':     433332 + 461299,  #  0-04 + 05-09
    '10_19':   507646 + 549033,  # 10-14 + 15-19
    '20_29':   544575 + 547505,  # 20-24 + 25-29
    '30_39':   571355 + 679093,  # 30-34 + 35-39
    '40_49':   792670 + 782555,  # 40-44 + 45-49
    '50_59':   747581 + 734540,  # 50-54 + 55-59
    '60_69':   672758 + 620543,  # 60-64 + 65-69
    '70_79':   544016 + 429107,  # 70-74 + 75-79
    '80_plus': 352218 + 316442,  # 80-84 + 85 ou mais
}


In [ ]:
# read data and reindex by date, for plots
data = pd.read_csv(Path.cwd() / '..' / '..' / 'data.csv', parse_dates=['data'], index_col='data', dayfirst=True)
data.tail(1).index


In [ ]:
df = data.copy()

AJUSTE = 1 # duplica incidencia7 

# 14 day accumulated, and per 100k and population
df['confirmados14'] = df.confirmados.diff(14)
df['incidencia'] = df.confirmados14 * 100_000 / POP_PT
df['confirmados_7'] = df.confirmados.diff(7)
df['incidencia_7'] = df.confirmados_7 * 100_000 / POP_PT * AJUSTE

for i in POP_ARS.keys():
  # per region, 14 day accumulated, and per 100k and population
  ii = i if i in ['acores', 'madeira'] else f'ars{i}'
  df[f'confirmados_{i}_14'] = df[f'confirmados_{ii}'].diff(14)
  df[f'incidencia_{i}'] = df[f'confirmados_{i}_14'] * 100_000 / POP_ARS[i]
  df[f'confirmados_{i}_7'] = df[f'confirmados_{ii}'].diff(7)
  df[f'incidencia_{i}_7'] = df[f'confirmados_{i}_7'] * 100_000 / POP_ARS[i] * AJUSTE

for i in POP_IDADE.keys():
  # por idade, 14 day accumulated, and per 100k and population
  ii = i
  df[f'confirmados_{i}'] = df[f'confirmados_{ii}_m'] + df[f'confirmados_{ii}_f']
  df[f'confirmados_{i}_14'] = df[f'confirmados_{ii}'].diff(14)
  df[f'incidencia_{i}'] = df[f'confirmados_{i}_14'] * 100_000 / POP_IDADE[i]
  df[f'confirmados_{i}_7'] = df[f'confirmados_{ii}'].diff(7)
  df[f'incidencia_{i}_7'] = df[f'confirmados_{i}_7'] * 100_000 / POP_IDADE[i] * AJUSTE

# discard first 13 days
df = df[ df.confirmados14.notnull() ]
df_incidencia = df


In [ ]:
df_incidencia_max = df_incidencia[[col for col in df_incidencia.columns if 'incidencia' in col and not '_7' in col]].max()
incidencia_max = df_incidencia_max.max()
print(incidencia_max)
df_incidencia_max


In [ ]:
def foo(k, n=None, df=df, days=14):
  BOOM = incidencia_max
  #BOOM = 1000 # 2000
  fig, axes = plt.subplots(figsize=(15,8))

  k = k if days == 14 else f"{k}_{days}"
  i = 960 * days / 14
  axes.fill_between(df.index, 0, df[k], color="#592212", label=f">{i/1}")
  axes.fill_between(df.index, 0, df[k], where=df[k] < i/1, color="#700", label=f"{i/2} a <{i/1}")
  axes.fill_between(df.index, 0, df[k], where=df[k] < i/2, color="red", label=f"{i/4} a <{i/2}")
  axes.fill_between(df.index, 0, df[k], where=df[k] < i/4, color="orange", label=f"{i/8} a <{i/4}")
  axes.fill_between(df.index, 0, df[k], where=df[k] < i/8, color="yellow", label=f"{i/16} a <{i/8}")
  axes.fill_between(df.index, 0, df[k], where=df[k] < i/16, color="green", label=f"<{i/16}")

  axes.axhline(linewidth=2, ls="--", color='#592212', y=i/1)
  axes.axhline(linewidth=2, ls="--", color='#700', y=i/2)
  axes.axhline(linewidth=2, ls="--", color='red', y=i/4)
  axes.axhline(linewidth=2, ls="--", color='orange', y=i/8)
  axes.axhline(linewidth=2, ls="--", color='yellow', y=i/16)
  #axes.axhline(linewidth=2, ls="--", color='green', y=0)

  prev=-1
  last = float(df[k][prev])
  while np.isnan(last):
    prev -= 1
    last = float(df[k][prev])

  axes.axhline(linewidth=2, ls="--", color='white', y=last)

  axes.legend()
  plt.xticks(list(reversed(list(reversed(list(df.index)))[0::7])), rotation=90)

  suffix = "" if days == 14 else f"_{days}"
  more = "" if days == 14 else f" (equivalente a {round(last*2, 1)} para 14 dias)"
  plot(axes, title=f'Incidência {days} dias para {n or k}: {round(last, 1)}{more}', file=f"../../temp/incidencia_{n or k}{suffix}.png", ylim=(0, BOOM * days / 14)).show()


In [ ]:
foo('incidencia', 'Nacional', df_incidencia)
foo('incidencia', 'Nacional', df_incidencia, 7)


In [ ]:
for k in POP_ARS.keys():
  foo(f'incidencia_{k}', k, df_incidencia)
  foo(f'incidencia_{k}', k, df_incidencia, 7)
  print()


In [ ]:
for k in POP_IDADE.keys():
  foo(f'incidencia_{k}', k, df_incidencia)
  foo(f'incidencia_{k}', k, df_incidencia, 7)
  print()


In [ ]:
df = df_incidencia
for k in POP_ARS.keys():
  mask = (df[f'incidencia_{k}'] >= 120)
  q = df[mask]
  q = q[[f'incidencia_{k}']]
  q['region'] = k
  print(q.head(1).append(q.tail(1)))
